# 📊 Step 1: Exploring the MedMCQA Dataset

**Goal**: Understand the data we'll use to fine-tune Nemotron-3-Nano.

Before training any model, you need to understand:
1. What does the data look like?
2. How is it structured?
3. How do we convert it to the format the model expects?

This is like understanding your input data format before writing a parser.

## 1.1 Load the Dataset

The `datasets` library from Hugging Face makes it easy to load public datasets.
Think of it like a package manager for ML datasets.

In [8]:
from datasets import load_dataset

# Load MedMCQA dataset from Hugging Face
# This downloads and caches the data (~500MB)
dataset = load_dataset("openlifescienceai/medmcqa")

# What did we get?
print(type(dataset))
print(dataset)

<class 'datasets.dataset_dict.DatasetDict'>
DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
        num_rows: 182822
    })
    test: Dataset({
        features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
        num_rows: 6150
    })
    validation: Dataset({
        features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
        num_rows: 4183
    })
})


## 1.2 Understanding the Structure

A `DatasetDict` is like a dictionary containing different splits:
- `train`: Data used to train the model
- `validation`: Data used to check progress during training  
- `test`: Data used for final evaluation (never seen during training)

**Systems analogy**: Think of it as train=dev, validation=staging, test=production.

In [2]:
# How many examples in each split?
for split_name, split_data in dataset.items():
    print(f"{split_name}: {len(split_data):,} examples")

train: 182,822 examples
test: 6,150 examples
validation: 4,183 examples


## 1.3 Inspecting a Single Example

Let's look at one question to understand the data format.
This is crucial - you need to know exactly what fields are available.

In [3]:
# Get the first example from the training set
example = dataset["train"][0]

# Print each field
for key, value in example.items():
    print(f"{key}: {value}")
    print("-" * 50)

id: e9ad821a-c438-4965-9f77-760819dfa155
--------------------------------------------------
question: Chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma
--------------------------------------------------
opa: Hyperplasia
--------------------------------------------------
opb: Hyperophy
--------------------------------------------------
opc: Atrophy
--------------------------------------------------
opd: Dyplasia
--------------------------------------------------
cop: 2
--------------------------------------------------
choice_type: single
--------------------------------------------------
exp: Chronic urethral obstruction because of urinary calculi, prostatic hyperophy, tumors, normal pregnancy, tumors, uterine prolapse or functional disorders cause hydronephrosis which by definition is used to describe dilatation of renal pelvis and calculus associated with progressive atrophy of the kidney due to obstruction to the o

## 1.4 Understanding the Fields

| Field | Description |
|-------|-------------|
| `question` | The medical question text |
| `opa`, `opb`, `opc`, `opd` | The four answer options (A, B, C, D) |
| `cop` | Correct Option index (0=A, 1=B, 2=C, 3=D) |
| `exp` | Explanation for the correct answer |
| `subject_name` | Medical subject (Anatomy, Pharmacology, etc.) |
| `topic_name` | Specific topic within the subject |

**Key insight**: We need to convert `cop` (0-3) to a letter (A-D) when formatting.

## 1.5 Converting to Instruction Format

LLMs are trained on **instruction-following** data. We need to convert the raw MCQ format into a conversation format that the model understands.

**Target format** (ChatML-style, which Nemotron uses):
```
<|im_start|>system
You are a medical expert...<|im_end|>
<|im_start|>user
Question: ...<|im_end|>
<|im_start|>assistant
The correct answer is...<|im_end|>
```

This is like converting JSON to protobuf - different serialization for the same data.

In [10]:
def format_medmcqa_to_instruction(example):
    """
    Convert a MedMCQA example to instruction format.
    
    Input: Raw dataset row with question, options, correct answer
    Output: Dict with 'text' field containing formatted conversation
    """
    # Map index to letter
    idx_to_letter = {0: "A", 1: "B", 2: "C", 3: "D"}
    options_map = {0: example["opa"], 1: example["opb"], 2: example["opc"], 3: example["opd"]}
    
    correct_idx = example["cop"]
    correct_letter = idx_to_letter[correct_idx]
    correct_text = options_map[correct_idx]
    
    # Build the instruction prompt
    system_prompt = "You are a medical expert. Answer the multiple choice question by selecting the correct option and providing a brief explanation."
    
    user_prompt = f"""Question: {example["question"]}

A) {example["opa"]}
B) {example["opb"]}
C) {example["opc"]}
D) {example["opd"]}"""
    
    # Build assistant response
    explanation = example.get("exp", "")
    if explanation and explanation.strip():
        assistant_response = f"The correct answer is {correct_letter}) {correct_text}.\n\nExplanation: {explanation}"
    else:
        assistant_response = f"The correct answer is {correct_letter}) {correct_text}."
    
    # Format as ChatML (Nemotron's expected format)
    formatted_text = f"""<|im_start|>system
{system_prompt}<|im_end|>
<|im_start|>user
{user_prompt}<|im_end|>
<|im_start|>assistant
{assistant_response}<|im_end|>"""
    
    return {"text": formatted_text}

In [11]:
# Test the formatting function on our example
formatted = format_medmcqa_to_instruction(example)
print(formatted["text"])

<|im_start|>system
You are a medical expert. Answer the multiple choice question by selecting the correct option and providing a brief explanation.<|im_end|>
<|im_start|>user
Question: Chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma

A) Hyperplasia
B) Hyperophy
C) Atrophy
D) Dyplasia<|im_end|>
<|im_start|>assistant
The correct answer is C) Atrophy.

Explanation: Chronic urethral obstruction because of urinary calculi, prostatic hyperophy, tumors, normal pregnancy, tumors, uterine prolapse or functional disorders cause hydronephrosis which by definition is used to describe dilatation of renal pelvis and calculus associated with progressive atrophy of the kidney due to obstruction to the outflow of urine Refer Robbins 7yh/9,1012,9/e. P950<|im_end|>


## 1.6 Apply Formatting to Entire Dataset

Now we'll apply our formatting function to all examples.
The `.map()` function is like `Array.map()` in JavaScript or `stream.map()` in Java.

In [13]:
# We only need train and validation splits for fine-tuning
# - train: for training the LoRA adapter
# - validation: for monitoring eval_loss to detect overfitting
# - test: skipped (MedMCQA test set has cop=-1, answers withheld for competition)

from datasets import DatasetDict

# Select only the splits we need
working_dataset = DatasetDict({
    "train": dataset["train"],
    "validation": dataset["validation"]
})

# Filter out examples with invalid correct answer indices (cop not in 0-3)
def is_valid_example(example):
    """Filter out examples where cop is not a valid index (0-3)."""
    return example["cop"] in [0, 1, 2, 3]

# Check how many invalid examples exist per split
for split_name, split_data in working_dataset.items():
    invalid_count = sum(1 for ex in split_data if ex["cop"] not in [0, 1, 2, 3])
    total = len(split_data)
    print(f"{split_name}: {invalid_count:,}/{total:,} invalid examples ({100*invalid_count/total:.1f}%)")

# Filter the dataset
filtered_dataset = working_dataset.filter(is_valid_example, desc="Filtering invalid examples")

print(f"\nAfter filtering:")
for split_name, split_data in filtered_dataset.items():
    print(f"  {split_name}: {len(split_data):,} examples")

# Now apply formatting
formatted_dataset = filtered_dataset.map(
    format_medmcqa_to_instruction,
    remove_columns=filtered_dataset["train"].column_names,
    desc="Formatting dataset"
)

print(f"\n{formatted_dataset}")
print("\nSample formatted example:")
print(formatted_dataset["train"][0]["text"][:500] + "...")

train: 0/182,822 invalid examples (0.0%)
validation: 0/4,183 invalid examples (0.0%)


Filtering invalid examples: 100%|██████████| 4183/4183 [00:00<00:00, 97415.76 examples/s]



After filtering:
  train: 182,822 examples
  validation: 4,183 examples


Formatting dataset: 100%|██████████| 4183/4183 [00:00<00:00, 8674.50 examples/s]


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 182822
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 4183
    })
})

Sample formatted example:
<|im_start|>system
You are a medical expert. Answer the multiple choice question by selecting the correct option and providing a brief explanation.<|im_end|>
<|im_start|>user
Question: Chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma

A) Hyperplasia
B) Hyperophy
C) Atrophy
D) Dyplasia<|im_end|>
<|im_start|>assistant
The correct answer is C) Atrophy.

Explanation: Chronic urethral obstruction because of urinary calculi, prostat...


## 1.7 Check Text Lengths

It's important to understand how long our examples are.
This affects:
- Memory usage during training
- Whether we need to truncate or pad
- Training speed

In [15]:
# Analyze text lengths (characters)
import numpy as np

lengths = [len(x["text"]) for x in formatted_dataset["train"]]

print(f"Character length statistics:")
print(f"  Min: {min(lengths):,}")
print(f"  Max: {max(lengths):,}")
print(f"  Mean: {np.mean(lengths):,.0f}")
print(f"  Median: {np.median(lengths):,.0f}")
print(f"  95th percentile: {np.percentile(lengths, 95):,.0f}")

Character length statistics:
  Min: 288
  Max: 22,941
  Mean: 914
  Median: 749
  95th percentile: 1,999


## 1.8 Save Formatted Dataset (Optional)

We can save the formatted dataset locally to avoid re-processing later.

In [16]:
# Save to disk (creates a directory with arrow files)
formatted_dataset.save_to_disk("../data/medmcqa_formatted")
print("✅ Dataset saved to ../data/medmcqa_formatted")

Saving the dataset (1/1 shards): 100%|██████████| 4183/4183 [00:00<00:00, 797888.65 examples/s]

✅ Dataset saved to ../data/medmcqa_formatted


---

## ✅ Summary

In this notebook, we:

1. **Loaded** the MedMCQA dataset from Hugging Face
2. **Explored** the structure (train/validation/test splits)
3. **Understood** the fields (question, options, correct answer, explanation)
4. **Created** a formatting function to convert MCQ → instruction format
5. **Applied** the transformation to all examples
6. **Analyzed** text lengths to understand our data
7. **Saved** the formatted dataset for later use

## ⏭️ Next Step

In the next notebook (`02_model_loading.ipynb`), we'll:
- Load the Nemotron-3-Nano base model
- Set up LoRA configuration
- Understand how the model processes our formatted text